In [1]:
import torch 
from pyvi import ViTokenizer
from rage import SentenceEmbedding

c:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Initialize Model

In [3]:
model= SentenceEmbedding(model_name= "vinai/phobert-base-v2", torch_dtype= torch.float32, aggregation_hidden_states= False, 
                         strategy_pooling= "dense_first")

c:\Users\Admin\anaconda3\envs\llm\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.to(device)

SentenceEmbeddingConfig(model_base: {'model_type_base': 'RobertaModel', 'model_name': 'vinai/phobert-base-v2', 'type_backbone': 'mlm', 'required_grad_base_model': True, 'aggregation_hidden_states': False, 'concat_embeddings': False, 'dropout': 0.1, 'quantization_config': None}, pooling: {'strategy_pooling': 'dense_first'})

In [8]:
model.load("best_sup_general_embedding_phobert2.pt", key= False)

In [9]:
model.summary_params()

trainable params: 135588864 || all params: 135588864 || trainable%: 100.0

In [10]:
model.summary()

+---------------------------+-------------+------------------+
|        Layer (type)       |    Params   | Trainable params |
+---------------------------+-------------+------------------+
|    model (RobertaModel)   | 134,998,272 |    134998272     |
| pooling (PoolingStrategy) |   590,592   |      590592      |
|       drp1 (Dropout)      |      0      |        0         |
+---------------------------+-------------+------------------+


In [12]:
sentences= [
    "Tôi đang đi học", 
    "Bạn tên là gì?",
]

sentences= list(map(lambda x: ViTokenizer.tokenize(x), sentences))

# return_tensors= "pt" -> output is torch.Tensor
model.encode(sentences, batch_size= 1, normalize_embedding= "l2", return_tensors= "np", verbose= 1)

1/2 [==============>...............] - ETA: 0s

2/2 [==============================] - 0s 43ms/Sample


array([[ 0.00281098, -0.00829096, -0.01582766, ...,  0.00878178,
         0.01830498, -0.00459659],
       [ 0.00249859, -0.03076724,  0.00033016, ...,  0.01299141,
        -0.00984358, -0.00703243]], dtype=float32)

In [ ]:
## model.push_to_hub("your repo")

### Load model from Huggingface Hub

In [13]:
model= SentenceEmbedding.from_pretrained('anti-ai/VieSemantic-base')

config_model.json: 100%|██████████| 355/355 [00:00<00:00, 352kB/s]
c:\Users\Admin\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaModel were not initialized from the model checkpoi

In [14]:
sentences = ["Nó rất thú_vị",
             "Nó không thú_vị ."]

output= model.encode(sentences, batch_size= 1, return_tensors= 'pt')

torch.cosine_similarity(output[0].view(1, -1), output[1].view(1, -1)).cpu().tolist()

2/2 [==============================] - 0s 40ms/Sample


[0.5605039596557617]